In [2]:
import os
import numpy as np
import cv2
import rasterio
from rasterio.shutil import copy
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape

In [4]:
base_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\FLNN\AAA_NewDataset_3\\"

# Define directories
dir_a = r"D:\RECTDNN\processing\2024-11-15_09-31-24\CornerTest\referenced\\"  # Directory A with referenced imagery
dir_b = f"{base_dir}000_drawn_double\\"  # Directory B with unreferenced imagery
dir_c = r"D:\RECTDNN\000_AGU_2\\"  # Output directory

# Ensure the output directory exists
os.makedirs(dir_c, exist_ok=True)

# Iterate through files in Directory A
for file_a in os.listdir(dir_a):
    file_a_path = os.path.join(dir_a, file_a)
    
    # Ensure the file is a raster
    if not file_a_path.lower().endswith(('.tif', '.tiff', '.img', '.vrt')):
        continue

    # Match files in Directory B based on name (ignoring extensions)
    base_name = os.path.splitext(file_a)[0]
    matching_files_b = [f for f in os.listdir(dir_b) if os.path.splitext(f)[0] == base_name]

    for file_b in matching_files_b:
        file_b_path = os.path.join(dir_b, file_b)

        # Load georeferencing information from Directory A
        with rasterio.open(file_a_path) as src_a:
            profile_a = src_a.profile

        # Open the image from Directory B
        with rasterio.open(file_b_path) as src_b:
            data = src_b.read()

        # Update the profile with referencing information from Directory A
        profile_b = profile_a.copy()
        profile_b.update(dtype=data.dtype, count=data.shape[0], height=data.shape[1], width=data.shape[2])

        # Save the updated raster to Directory C
        output_path = os.path.join(dir_c, file_b.split(".")[0]+ ".tif")
        with rasterio.open(output_path, 'w', **profile_b) as dst:
            dst.write(data)

print("Georeferencing transfer complete.")


Georeferencing transfer complete.


In [6]:
def threshold_and_polygonize(input_raster, output_vector, threshold_value):
    """
    Applies thresholding and polygonizes a raster.

    Parameters:
    - input_raster (str): Path to the input raster file.
    - output_vector (str): Path to the output vector file (GeoJSON).
    - threshold_value (int or float): Threshold value for binary mask creation.

    Returns:
    - None
    """
    with rasterio.open(input_raster) as src:
        # Read the first band
        data = src.read(1)
        transform = src.transform

        # Apply threshold to create a binary mask
        binary_mask = (data > threshold_value).astype(rasterio.uint8)

        # Polygonize the binary mask
        shapes_generator = shapes(binary_mask, transform=transform)
        polygons = [
            shape(geom) for geom, value in shapes_generator if value == 1
        ]

        # Create GeoDataFrame
        gdf = gpd.GeoDataFrame(geometry=polygons, crs=src.crs)

    # Save polygons to the output vector file
    gdf.to_file(output_vector, driver="GeoJSON")
    print(f"Polygonized output saved to: {output_vector}")

def process_directory(input_dir, output_dir, threshold_value):
    """
    Processes all raster files in a directory and saves polygonized results.

    Parameters:
    - input_dir (str): Path to the input directory containing raster files.
    - output_dir (str): Path to the output directory for vector files.
    - threshold_value (int or float): Threshold value for binary mask creation.

    Returns:
    - None
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all files in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.tif', '.tiff')):
            input_raster = os.path.join(input_dir, filename)
            output_vector = os.path.join(output_dir, os.path.splitext(filename)[0] + ".geojson")

            # Apply thresholding and polygonization
            threshold_and_polygonize(input_raster, output_vector, threshold_value)


In [7]:
# input_directory =   r"D:\RECTDNN\000_AGU_2\\"
output_directory =  r"D:\RECTDNN\000_AGU_2\vectors\\"
threshold = 100

process_directory(dir_c, output_directory, threshold)

Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0845J.geojson
Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0845K.geojson
Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0855J.geojson
Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0855K.geojson
Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0865J.geojson
Polygonized output saved to: D:\RECTDNN\000_AGU_2\vectors\\48201C0865K.geojson
